In [9]:
import os
import sys

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from astropy.visualization import make_lupton_rgb

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='../../config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import pyhalo, pandeia_input

In [10]:
input_dir = os.path.join(config.machine.pipeline_dir, '04_test')

In [11]:
limit = 25
uid_list = [str(uid).zfill(8) for uid in list(range(limit))]

In [12]:
rng = np.random.default_rng()
image_r = rng.random((100,100))
image_g = rng.random((100,100))
image_b = rng.random((100,100))
image = make_lupton_rgb(image_r, image_g, image_b, stretch=0.5)
plt.imshow(image)

In [13]:
def _rescale_rgb_float(image_b, image_g, image_r):
    max = plot_util.get_v([image_b, image_g, image_r])
    image_b /= max
    image_g /= max
    image_r /= max

    return image_b, image_g, image_r

def get_rgb(image_b, image_g, image_r, minimum=None, stretch=3, Q=4):
    # image_b, image_g, image_r = _rescale_rgb_float(image_b, image_g, image_r)

    # assert image_b.shape == image_g.shape == image_r.shape
    if minimum is None:
        minimum = np.min(np.concatenate((image_b, image_g, image_r)))
    return make_lupton_rgb(image_r=image_r, image_g=image_g, image_b=image_b, minimum=minimum, stretch=stretch, Q=Q)

In [14]:
color_images = []

for uid in uid_list:
    f106 = np.load(input_dir + f'/galsim_{uid}_F106.npy')
    f129 = np.load(input_dir + f'/galsim_{uid}_F129.npy')
    f184 = np.load(input_dir + f'/galsim_{uid}_F184.npy')

    rgb_image = get_rgb(image_b=f106, image_g=f129, image_r=f184)
    color_images.append(rgb_image)

In [15]:
print(color_images[0].shape)

(45, 45, 3)


In [16]:
test = color_images[0]